# Task

Para la entrega de esta práctica, la tarea consiste en desarrollar un prototipo que procese uno (vídeo ejemplo proporcionado) o varios vídeos (incluyendo vídeos de cosecha propia):

- Detecte y siga las personas y vehículos presentes
- Detecte y lea las matrículas de los vehículos presentes
- Cuente el total de cada clase
- Vuelque a disco un vídeo que visualice los resultados
- Genere un archivo csv con el resultado de la detección y seguimiento. Se sugiere un formato con al menos los siguientes campos:

`fotograma, tipo_objeto, confianza, identificador_tracking, x1, y1, x2, y2, matrícula_en_su_caso, confianza, mx1,my1,mx2,my2, texto_matricula`

In [ ]:
# Import necessary libraries
from collections import defaultdict
from pytesseract import Output
from ultralytics import YOLO
import pytesseract
import numpy as np
import math
import cv2

In [ ]:
# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, especificamos la ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# Variable booleana que indica si quieres guardar el video en memoria (False) o visualizar en pantalla (True)
display = True

# Carga del modelo YOLO para detección de objetos, descargando el archivo del modelo si no está disponible localmente
model = YOLO('YOLO/yolo11n.pt')  # Modelo para detectar vehículos
license_plate_model = YOLO('YOLO/best.pt')  # Modelo para detectar matrículas

# Etiquetas de las distintas clases de objetos
classNames = ["person", "bicycle", "car", "motorbike", "", "bus", "", "truck"]

# Configuración para la captura de video desde un archivo
vid = cv2.VideoCapture("Videos/Video 3.mp4")

# Si se guarda en disco, establece el ancho, alto y configuración del archivo de salida
if not display:
    frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_video = cv2.VideoWriter('YOLO/resultados.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))

# Ciclo para procesar cada fotograma del video
while(True):      
    # Lee el siguiente fotograma del video
    ret, img = vid.read()

    # Si el fotograma es válido
    if ret:  
        # Realiza el seguimiento de objetos con persistencia entre fotogramas, limitando a ciertas clases de interés
        results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])
        
        # Procesa cada resultado de detección
        for r in results:
            boxes = r.boxes

            # Procesa cada caja de delimitación de objeto detectado
            for box in boxes:
                # Define los límites de la caja de delimitación
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Obtiene el ID de seguimiento si está disponible
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Calcula el nivel de confianza de la detección
                confidence = math.ceil((box.conf[0] * 100)) / 100
                print("Confianza --->", confidence)

                # Determina la clase de objeto
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Si la clase detectada es vehículo (por ejemplo, clase >= 2 en este contexto)
                if cls >= 2:
                    # Recorta la imagen del contenedor del vehículo detectado
                    car_crop = img[y1:y2, x1:x2]

                    # Detecta matrículas en el contenedor del vehículo
                    license_plate_results = license_plate_model.predict(car_crop)
                    possible_licenses = {"Not Found": 0}

                    # Procesa cada detección de matrícula en el vehículo
                    for lp_result in license_plate_results:
                        lp_boxes = lp_result.boxes
                        for lp_box in lp_boxes:
                            # Coordenadas de la caja de delimitación de la matrícula
                            lp_x1, lp_y1, lp_x2, lp_y2 = lp_box.xyxy[0]
                            lp_x1, lp_y1, lp_x2, lp_y2 = int(lp_x1), int(lp_y1), int(lp_x2), int(lp_y2)

                            # Usa OCR para extraer texto de la matrícula detectada y almacenar el posible valor
                            possible_licenses[pytesseract.image_to_string(img)] = 1

                            # Ajusta las coordenadas de la matrícula para el marco completo
                            lp_x1 += x1
                            lp_x2 += x1
                            lp_y1 += y1
                            lp_y2 += y1

                            # Dibuja el rectángulo de la matrícula y la muestra en la imagen
                            cv2.rectangle(img, (lp_x1, lp_y1), (lp_x2, lp_y2), (0, 255, 0), 2)
                            cv2.putText(img,
                                        f"Matricula: {max(possible_licenses, key=possible_licenses.get)}",
                                        (lp_x1, lp_y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Asigna un color RGB único según la clase para visualizar cada tipo de objeto detectado
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255 * 2:
                    R, G, B = 255, 255, escala - 255 * 2
                else:
                    R, G, B = (255, escala - 255, 0) if escala >= 255 else (escala, 0, 0)

                # Dibuja el rectángulo de la caja y el nombre de la clase en la imagen
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls], [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra el fotograma en pantalla o lo guarda en el archivo de video
        if display:
            cv2.imshow('Vid', img)
        else:
            output_video.write(img)

        # Detiene el proceso si se presiona la tecla ESC
        if cv2.waitKey(20) == 27:
            break
    else:
        break
  
# Libera el recurso de captura de video
vid.release()
if display:
    cv2.destroyAllWindows()
else:
    output_video.release()  # Libera el archivo de video